In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb

np.random.seed(2018)

trn = pd.read_csv('C:/santander-product-recommendation/train_ver2.csv')
tst = pd.read_csv('C:/santander-product-recommendation/test_ver2.csv')

C:\Users\User\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (5,8,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\User\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## 데이터 전처리

In [2]:
#데이터 전처리
prods = trn.columns[24:].tolist()

#제품 변수 결측값을 미리 0으로 대체
trn[prods] = trn[prods].fillna(0.0).astype(np.int8)

#24개 제품 중 하나도 보유하지 않는 고객 데이터 제거
no_product = trn[prods].sum(axis=1) == 0 
trn= trn[~no_product]

# 훈련 데이터와 테스트 데이터 통합. 테스트 데이터에 없는 제품 변수는 0으로 채운다
for col in trn.columns[24:]:
    tst[col]=0
df = pd.concat([trn, tst], axis = 0)

# 학습에 사용할 변수 담는 list
features = []

#범주형 변수를 .factorize() 함수 통해 label encoding한다.
categorical_cols = ['ind_empleado','pais_residencia','sexo','tiprel_1mes','indresi','indext','conyuemp','canal_entrada','indfall','tipodom','nomprov','segmento']
for col in categorical_cols:
    df[col], _ = df[col].factorize(na_sentinel=-99)
features += categorical_cols

# 수치형 변수의 특이값과 결측값을 -99로 대체하고, 정수형으로 변환한다
df['age'].replace(' NA', -99, inplace=True)
df['age'] = df['age'].astype(np.int8)

df['antiguedad'].replace('     NA', -99, inplace=True)
df['antiguedad'] = df['antiguedad'].astype(np.int8)

df['renta'].replace('         NA', -99, inplace=True)
df['renta'].fillna(-99, inplace=True)
df['renta'] = df['renta'].astype(float).astype(np.int8)


df['indrel_1mes'].replace('P',5,inplace = True)
df['indrel_1mes'].fillna(-99, inplace = True)
df['indrel_1mes'] = df['indrel_1mes'].astype(float).astype(np.int8)

# 학습에 사용할 수치형 변수를 features에 추구한다.
features += ['age','antiguedad','renta','ind_nuevo','indrel','indrel_1mes','ind_acitvidad_cliente']


## 피처 엔지니어링

### 위에서 데이터 전처리를 해주었으니, 피처도 만져준다. 
### 머신러닝 모델에 필요한 파생변수를 생성하는 과정

In [3]:
df['fecha_alta_month'] = df['fecha_alta'].map(lambda x:0.0 if x.__class__ is float else float(x.split('-')[1])).astype(np.int8)
df['fecha_alta_year'] = df['fecha_alta'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[0])).astype(np.int16)

features += ['fecha_alta_month','fecha_alta_year']

In [4]:
df['ult_fec_cli_1t_month'] = df['ult_fec_cli_1t'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[1])).astype(np.int8)
df['ult_fec_cli_1t_year'] = df['ult_fec_cli_1t'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[0])).astype(np.int16)

features += ['ult_fec_cli_1t_month', 'ult_fec_cli_1t_year']

In [8]:
# 그외 변수의 결측값은 모두 -99로 대체
df.fillna(-99, inplace = True)

def date_to_int(str_date):
    Y, M, D = [int(a) for a in str_date.strip().split('-')]
    int_date = (int(Y)-2015) * 12 + int(M)
    return int_date


# 날짜를 숫자로 변환하여 int_date에 저장한다
df['int_date'] = df['fecha_dato'].map(date_to_int).astype(np.int8)

# 데이터를 복사하고, int_date날짜에 1을 더하여 lag를 생성한다. 변수명에 _prev를 추가한다.
df_lag = df.copy()
df_lag.columns = [col+'_prev' if col not in ['ncodpers','int_date'] else col for col in df.columns]
df_lag['int_date'] += 1

# 원본 데이터와 lag 데이터를 ncodper와 int_date 기준으로 합친다. lag 데이터의 int_date는 1 밀려 있기 때문에, 저번 달의 제품 정보가 삽입된다.
df_trn = df.merge(df_lag, on=['ncodpers','int_date'], how = 'left')

del df, df_lag


MemoryError: 